In [1]:
from sage.all import *
import pickle

sx, sy = var('sx, sy')

ox, oy = var('ox, oy')

dx = ox - sx
dy = oy - sy
r2 = (dx ** 2) + (dy ** 2)

r = sqrt(r2)
G = (1 / (2 * pi)) * log(r)

In [2]:
forget()
assume(sy-oy>0)
assume(sy > 0)
assume(sy < 1)
Ia = integral(G, sx, -1, 1)

In [3]:
assume(ox > -1)
assume(ox < 1)
assume(oy > -1)
coincident = integral(Ia, sy, -1, 1)

In [4]:
forget()
assume(ox > 1)
assume(oy > 1)
nearfield = integral(Ia, sy, -1, 1)

In [5]:
import pickle
with open('data/constant_test_integral.pkl', 'wb') as f:
    pickle.dump([coincident._sympy_(), nearfield._sympy_()], f)

In [10]:
forget()
assume(sy-oy>0)
assume(sy > 0)
assume(sy < 1)
Ia = integral(G * (1 - sx) * (1 - sy ** 2), sx, -1, 1)

In [11]:
assume(ox > -1)
assume(ox < 1)
assume(oy > -1)
coincident = integral(Ia, sy, -1, 1)

In [12]:
forget()
assume(ox > 1)
assume(oy > 1)
nearfield = integral(Ia, sy, -1, 1)

In [13]:
import pickle
with open('data/xy_test_integral.pkl', 'wb') as f:
    pickle.dump([coincident._sympy_(), nearfield._sympy_()], f)